In [1]:
import sys,os
sys.path.append(os.path.expanduser('~/smodels'))
from smodels.tools import runtime
# Define your model (list of BSM particles)
runtime.modelFile = 'smodels.share.models.mssm'
# runtime.modelFile = 'mssmQNumbers.slha'

from smodels.theory import decomposer
from smodels.tools.physicsUnits import fb, GeV, TeV
from smodels.theory.theoryPrediction import theoryPredictionsFor
from smodels.experiment.databaseObj import Database
from smodels.tools.theoryPredictionsCombiner import TheoryPredictionsCombiner
from smodels.tools.smodelsLogging import setLogLevel
from smodels.particlesLoader import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.theory.model import Model
from smodels.theory.branch import Branch
from smodels.tools.coverage import Uncovered,UncoveredGroup,GeneralElement
from smodels.theory.auxiliaryFunctions import index_bisect
%load_ext line_profiler
setLogLevel("error")

# Set the path to the database
database = Database(os.path.expanduser("~/smodels/test/database"))

### Load model

In [2]:
slhafile = '../inputFiles/slha/lightEWinos.slha'
model = Model(BSMparticles=BSMList, SMparticles=SMList)
model.updateParticles(inputFile=slhafile)


### Decompose

In [3]:
# Set main options for decomposition
sigmacut = 10*fb
mingap = 5.*GeV

# Decompose model
topDict = decomposer.decompose(model, sigmacut,
                               doCompress=True, doInvisible=True,
                               minmassgap=mingap)

In [13]:
print(len(topDict.getElements()))
print(topDict.getTotalWeight())

105
['8.00E+00 [TeV]:3.74E+00 [pb] (None, None)', '1.30E+01 [TeV]:8.40E+00 [pb] (None, None)']


### Load the experimental results to be used.

In [5]:
listOfExpRes = database.getExpResults()

### Compute the theory predictions

In [6]:
print("\n Theory Predictions and Constraints:")
rmax = 0.
bestResult = None
allPredictions = []
for expResult in listOfExpRes:
    predictions = theoryPredictionsFor(expResult, topDict, combinedResults=False, marginalize=False)
    if not predictions:
        continue  # Skip if there are no constraints from this result
    print('\n %s ' % expResult.globalInfo.id)
    for theoryPrediction in predictions:
        # Compute the r-value
        r = theoryPrediction.getRValue()
        print("r = %1.3E" % r)


 Theory Predictions and Constraints:

 ATLAS-CONF-2013-037 
r = 1.093E-01

 ATLAS-SUSY-2013-02 
r = 6.560E-05


In [7]:
uncovered = Uncovered(topDict, sqrts=13.*TeV, sigmacut=20*fb)

In [8]:
# First sort coverage groups by label
groups = sorted(uncovered.groups[:], key=lambda g: g.label)
# Print uncovered cross-sections:
for group in groups:
    print("\nTotal cross-section for %s (fb): %10.3E\n" % (group.description, group.getTotalXSec()))


Total cross-section for missing topologies (fb):  5.132E+03


Total cross-section for missing topologies with displaced decays (fb):  0.000E+00


Total cross-section for missing topologies with prompt decays (fb):  5.132E+03


Total cross-section for topologies outside the grid (fb):  4.719E+01



In [9]:
missingTopos = uncovered.getGroup('missing (prompt)')
# Print some of the missing topologies:
if missingTopos.generalElements:
    print('Missing topologies (up to 3):')
    for genEl in missingTopos.generalElements[:3]:
        print('Element:', genEl)
        print('\tcross-section (fb):', genEl.missingX)
else:
    print("No missing topologies found\n")

Missing topologies (up to 3):
Element: [[[jet,jet]],[[l,nu]]]  (MET,MET)
	cross-section (fb): 1203.688445123887
Element: [[[jet,jet]],[[nu,ta]]]  (MET,MET)
	cross-section (fb): 600.2542979266833
Element: [[[jet,jet]],[[b,b]]]  (MET,MET)
	cross-section (fb): 507.5376500009445


In [10]:
missingDisplaced = uncovered.getGroup('missing (displaced)')
# Print elements with displaced decays:
if missingDisplaced.generalElements:
    print('\nElements with displaced vertices (up to 2):')
    for genEl in missingDisplaced.generalElements[:2]:
        print('Element:', genEl)
        print('\tcross-section (fb):', genEl.missingX)
else:
    print("\nNo displaced decays")



No displaced decays


In [11]:
for genEl in missingTopos.generalElements[:1]:
    print('Element:', genEl)
    print('\tcross-section (fb):', genEl.missingX)
    for el in genEl._contributingElements:
        print(el,el.weight.getMaxXsec())
        print(el.oddParticles)


Element: [[[jet,jet]],[[l,nu]]]  (MET,MET)
	cross-section (fb): 1203.688445123887
[[[e+,nu]],[[q,q]]] 1.46E-01 [pb]
[[C1+, N1], [N2, N1]]
[[[mu+,nu]],[[q,q]]] 1.46E-01 [pb]
[[C1+, N1], [N2, N1]]
[[[e-,nu]],[[q,q]]] 8.28E-02 [pb]
[[C1-, N1~], [N2, N1]]
[[[mu-,nu]],[[q,q]]] 8.28E-02 [pb]
[[C1-, N1~], [N2, N1]]
[[[e-,nu]],[[q,q]]] 7.72E-02 [pb]
[[C1-, N1~], [C1+, N1]]
[[[e-,nu]],[[c,q]]] 7.72E-02 [pb]
[[C1-, N1~], [C1+, N1]]
[[[mu-,nu]],[[q,q]]] 7.72E-02 [pb]
[[C1-, N1~], [C1+, N1]]
[[[mu-,nu]],[[c,q]]] 7.72E-02 [pb]
[[C1-, N1~], [C1+, N1]]
[[[e+,nu]],[[q,q]]] 7.72E-02 [pb]
[[C1+, N1], [C1-, N1~]]
[[[e+,nu]],[[c,q]]] 7.72E-02 [pb]
[[C1+, N1], [C1-, N1~]]
[[[mu+,nu]],[[q,q]]] 7.72E-02 [pb]
[[C1+, N1], [C1-, N1~]]
[[[mu+,nu]],[[c,q]]] 7.72E-02 [pb]
[[C1+, N1], [C1-, N1~]]
[[[e+,nu]],[[c,c]]] 4.07E-02 [pb]
[[C1+, N1], [N2, N1]]
[[[mu+,nu]],[[c,c]]] 4.07E-02 [pb]
[[C1+, N1], [N2, N1]]
[[[e-,nu]],[[c,c]]] 2.30E-02 [pb]
[[C1-, N1~], [N2, N1]]
[[[mu-,nu]],[[c,c]]] 2.30E-02 [pb]
[[C1-, N1~], [N2,